In [ ]:
# Import section
import cv2
import numpy as np
from os import scandir, getcwd, rename
import requests
import time
import json
import re
import pandas as pd
from paralel_send import multi_send

In [18]:
# Getting arrays
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\areas_result\\'

m_widths = np.genfromtxt(ruta+'widths.txt')
m_heights = np.genfromtxt(ruta+'heights.txt')
m_cxs = np.genfromtxt(ruta+'cxs.txt')
m_cys = np.genfromtxt(ruta+'cys.txt')
m_files = np.load(ruta+'paths.npy')
m_azure_flags = np.load(ruta+'azure_flags.npy')

m_files.size

50000

In [34]:
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\temps_fail\\'
files = []
jsons = []
for i in range(10):
    file = np.load(ruta + 'n_files_{}.npy'.format(i), allow_pickle=True)
    azu = np.load(ruta + 'n_jsons_{}.npy'.format(i), allow_pickle=True)
    if file.size > 0 and azu.size > 0:
        files.append(file)
        jsons.append(azu)
azure_files = np.concatenate(files[:], axis=0)
azure_jsons = np.concatenate(jsons[:], axis=0)

azure_files.shape , azure_jsons.shape

((50000,), (50000,))

In [35]:
m_files.shape

(50000,)

In [36]:
azure_files.shape

(50000,)

In [37]:
pend = np.setdiff1d(m_files, azure_files)

In [38]:
pend[:]

array([], dtype='<U84')

In [28]:
def treat_path(path_str):
    name = path_str.split('\\')[-1]
    return ('D:\\GyS\\proyectos\\cuponesWong\\proceso_definitivo\\process\\m_get50k\\'+name)

def des_treat_path(path_str):
    name = path_str.split('\\')[-1]
    return ('E:\\git\\cupones_wong\\notebooks_flow\\input\\'+name)

n_treat_path = np.vectorize(treat_path)
n_des_treat_path = np.vectorize(des_treat_path)

In [10]:
treat = n_treat_path(pend)

In [11]:
treat.shape

(5000,)

In [12]:
NUM_WORKERS = 10
print('Empezó multi send')
multi_send(treat[:].tolist(), NUM_WORKERS, 1, 1)
print('Terminó multi send')

Empezó multi send
Tamaño por worker: 500 de total 5000
Existe un extra de 0 a repartir
Worker 0: Reading 0.MarzoCompras_LV_B4_20190708113258_00054.jpg
Worker 1: Reading 0.MarzoCompras_LV_B4_20190708124105_00768.jpg
Worker 2: Reading 0.MarzoCompras_LV_B4_20190708164702_00465.jpg
Worker 3: Reading 0.MarzoCompras_MD_B2_20190704095258_00071.jpgWorker 4: Reading 0.MarzoCompras_MD_B2_20190704102515_00888.jpg

Worker 5: Reading 0.MarzoCompras_MD_B2_20190704112224_00661.jpgWorker 6: Reading 0.MarzoCompras_MD_B2_20190704123653_00384.jpg

Worker 7: Reading 0.MarzoCompras_MD_B2_20190704143331_00162.jpg
Worker 8: Reading 0.MarzoCompras_MD_B2_20190704150000_00883.jpg
Worker 9: Reading 0.MarzoCompras_MD_B2_20190704172514_00571.jpg
Worker 4: Status:{'status': 'Succeeded', '
Worker 4: Finished 0.MarzoCompras_MD_B2_20190704102515_00888.jpg and waiting for 1
Worker 3: Status:{'status': 'Succeeded', '
Worker 3: Finished 0.MarzoCompras_MD_B2_20190704095258_00071.jpg and waiting for 1
Worker 1: Status:{'st

Worker 4: Reading 4.MarzoCompras_MD_B2_20190704102516_00916.jpg
Worker 6: Status:{'status': 'Succeeded', '
Worker 6: Finished 4.MarzoCompras_MD_B2_20190704123653_00391.jpg and waiting for 1
Worker 3: Status:{'status': 'Succeeded', '
Worker 3: Finished 4.MarzoCompras_MD_B2_20190704095258_00085.jpg and waiting for 1
Worker 1: Status:{'status': 'Succeeded', '
Worker 1: Finished 4.MarzoCompras_LV_B4_20190708124106_00787.jpg and waiting for 1
Worker 0: Status:{'status': 'Succeeded', '
Worker 0: Finished 4.MarzoCompras_LV_B4_20190708113258_00063.jpg and waiting for 1
Worker 9: Status:{'status': 'Succeeded', '
Worker 9: Finished 4.MarzoCompras_MD_B2_20190704172515_00583.jpg and waiting for 1
Worker 2: Status:{'status': 'Succeeded', '
Worker 2: Finished 4.MarzoCompras_LV_B4_20190708164702_00475.jpg and waiting for 1
Worker 8: Status:{'status': 'Succeeded', '
Worker 8: Finished 4.MarzoCompras_MD_B2_20190704150000_00894.jpg and waiting for 1
Worker 6: Reading 5.MarzoCompras_MD_B2_20190704123653_

Worker 9: Status:{'status': 'Succeeded', '
Worker 9: Finished 8.MarzoCompras_MD_B2_20190704172515_00600.jpg and waiting for 1
Worker 7: Status:{'status': 'Succeeded', '
Worker 7: Finished 8.MarzoCompras_MD_B2_20190704143332_00185.jpg and waiting for 1
Worker 3: Reading 9.MarzoCompras_MD_B2_20190704095259_00110.jpg
Worker 2: Status:{'status': 'Succeeded', '
Worker 2: Finished 8.MarzoCompras_LV_B4_20190708164704_00497.jpg and waiting for 1
Worker 1: Reading 9.MarzoCompras_LV_B4_20190708124106_00805.jpg
Worker 6: Reading 9.MarzoCompras_MD_B2_20190704123653_00402.jpg
Worker 5: Status:{'status': 'Succeeded', '
Worker 5: Finished 8.MarzoCompras_MD_B2_20190704112225_00689.jpg and waiting for 1
Worker 0: Reading 9.MarzoCompras_LV_B4_20190708113259_00078.jpg
Worker 8: Reading 9.MarzoCompras_MD_B2_20190704150000_00907.jpg
Worker 9: Reading 9.MarzoCompras_MD_B2_20190704172515_00606.jpg
Worker 7: Reading 9.MarzoCompras_MD_B2_20190704143332_00188.jpg
Worker 2: Reading 9.MarzoCompras_LV_B4_201907081

Worker 9: Reading 13.MarzoCompras_MD_B2_20190704172516_00620.jpg
Worker 7: Reading 13.MarzoCompras_MD_B2_20190704143332_00195.jpg
Worker 0: Reading 13.MarzoCompras_LV_B4_20190708113259_00089.jpg
Worker 5: Status:{'status': 'Succeeded', '
Worker 5: Finished 12.MarzoCompras_MD_B2_20190704112226_00701.jpg and waiting for 1
Worker 8: Reading 13.MarzoCompras_MD_B2_20190704150001_00915.jpg
Worker 4: Status:{'status': 'Succeeded', '
Worker 4: Finished 12.MarzoCompras_MD_B2_20190704102517_00949.jpg and waiting for 1
Worker 2: Reading 13.MarzoCompras_LV_B4_20190708164707_00522.jpg
Worker 5: Reading 13.MarzoCompras_MD_B2_20190704112226_00702.jpg
Worker 4: Reading 13.MarzoCompras_MD_B2_20190704102517_00956.jpg
Worker 3: Status:{'status': 'Succeeded', '
Worker 3: Finished 13.MarzoCompras_MD_B2_20190704095259_00116.jpg and waiting for 1
Worker 1: Status:{'status': 'Succeeded', '
Worker 1: Finished 13.MarzoCompras_LV_B4_20190708124107_00813.jpg and waiting for 1
Worker 6: Status:{'status': 'Succeede

Worker 3: Reading 18.MarzoCompras_MD_B2_20190704095300_00139.jpg
Worker 5: Reading 17.MarzoCompras_MD_B2_20190704112226_00716.jpg
Worker 4: Reading 17.MarzoCompras_MD_B2_20190704102518_00975.jpg
Worker 7: Reading 17.MarzoCompras_MD_B2_20190704143332_00205.jpg
Worker 0: Reading 17.MarzoCompras_LV_B4_20190708113259_00100.jpg
Worker 9: Reading 17.MarzoCompras_MD_B2_20190704172516_00642.jpg
Worker 1: Status:{'status': 'Succeeded', '
Worker 1: Finished 17.MarzoCompras_LV_B4_20190708124107_00822.jpg and waiting for 1
Worker 6: Status:{'status': 'Succeeded', '
Worker 6: Finished 17.MarzoCompras_MD_B2_20190704123654_00429.jpg and waiting for 1
Worker 1: Reading 18.MarzoCompras_LV_B4_20190708124107_00825.jpg
Worker 2: Status:{'status': 'Succeeded', '
Worker 2: Finished 17.MarzoCompras_LV_B4_20190708164707_00531.jpg and waiting for 1
Worker 8: Status:{'status': 'Succeeded', '
Worker 8: Finished 17.MarzoCompras_MD_B2_20190704150001_00928.jpg and waiting for 1
Worker 6: Reading 18.MarzoCompras_MD_

...

Worker 3: Status:{'status': 'Succeeded', '
Worker 3: Finished 483.MarzoCompras_MD_B2_20190704102514_00845.jpg and waiting for 1
Worker 8: Reading 483.MarzoCompras_MD_B2_20190704172513_00522.jpg
Worker 4: Status:{'status': 'Succeeded', '
Worker 4: Finished 480.MarzoCompras_MD_B2_20190704112222_00581.jpg and waiting for 1
Worker 0: Reading 482.MarzoCompras_LV_B4_20190708124103_00708.jpg
Worker 6: Reading 484.MarzoCompras_MD_B2_20190704143330_00128.jpg
Worker 7: Status:{'status': 'Succeeded', '
Worker 7: Finished 480.MarzoCompras_MD_B2_20190704145958_00809.jpg and waiting for 1
Worker 9: Status:{'status': 'Succeeded', '
Worker 9: Finished 479.MarzoCompras_MD_B2_20190705105250_00184.jpg and waiting for 1
Worker 3: Reading 484.MarzoCompras_MD_B2_20190704102514_00846.jpg
Worker 5: Status:{'status': 'Succeeded', '
Worker 5: Finished 481.MarzoCompras_MD_B2_20190704123651_00308.jpg and waiting for 1
Worker 4: Reading 481.MarzoCompras_MD_B2_20190704112222_00587.jpg
Worker 7: Reading 481.MarzoCom

Worker 4: Status:{'status': 'Succeeded', '
Worker 4: Finished 484.MarzoCompras_MD_B2_20190704112222_00600.jpg and waiting for 1
Worker 3: Reading 488.MarzoCompras_MD_B2_20190704102514_00857.jpg
Worker 6: Reading 488.MarzoCompras_MD_B2_20190704143331_00138.jpg
Worker 7: Reading 485.MarzoCompras_MD_B2_20190704145958_00824.jpg
Worker 9: Reading 484.MarzoCompras_MD_B2_20190705105251_00210.jpg
Worker 5: Status:{'status': 'Succeeded', '
Worker 5: Finished 485.MarzoCompras_MD_B2_20190704123651_00323.jpg and waiting for 1
Worker 4: Reading 485.MarzoCompras_MD_B2_20190704112222_00602.jpg
Worker 5: Reading 486.MarzoCompras_MD_B2_20190704123651_00325.jpg
Worker 1: Status:{'status': 'Succeeded', '
Worker 1: Finished 482.MarzoCompras_LV_B4_20190708164657_00409.jpg and waiting for 1
Worker 2: Status:{'status': 'Succeeded', '
Worker 2: Finished 476.MarzoCompras_LV_B5_20190709151149_00771.jpg and waiting for 1
Worker 8: Status:{'status': 'Succeeded', '
Worker 8: Finished 487.MarzoCompras_MD_B2_2019070

Worker 4: Reading 489.MarzoCompras_MD_B2_20190704112223_00614.jpg
Worker 1: Status:{'status': 'Succeeded', '
Worker 1: Finished 486.MarzoCompras_LV_B4_20190708164658_00420.jpg and waiting for 1
Worker 5: Reading 490.MarzoCompras_MD_B2_20190704123652_00351.jpg
Worker 2: Status:{'status': 'Succeeded', '
Worker 2: Finished 480.MarzoCompras_LV_B5_20190709151149_00779.jpg and waiting for 1
Worker 8: Status:{'status': 'Succeeded', '
Worker 8: Finished 491.MarzoCompras_MD_B2_20190704172514_00543.jpg and waiting for 1
Worker 0: Status:{'status': 'Succeeded', '
Worker 0: Finished 490.MarzoCompras_LV_B4_20190708124104_00739.jpg and waiting for 1
Worker 1: Reading 487.MarzoCompras_LV_B4_20190708164659_00428.jpg
Worker 6: Status:{'status': 'Succeeded', '
Worker 6: Finished 492.MarzoCompras_MD_B2_20190704143331_00148.jpg and waiting for 1
Worker 3: Status:{'status': 'Succeeded', '
Worker 3: Finished 492.MarzoCompras_MD_B2_20190704102514_00864.jpg and waiting for 1
Worker 2: Reading 481.MarzoCompras

Worker 1: Reading 491.MarzoCompras_LV_B4_20190708164700_00437.jpg
Worker 2: Reading 485.MarzoCompras_MD_B2_20190704095256_00019.jpg
Worker 3: Status:{'status': 'Succeeded', '
Worker 3: Finished 496.MarzoCompras_MD_B2_20190704102515_00873.jpg and waiting for 1
Worker 7: Status:{'status': 'Succeeded', '
Worker 7: Finished 493.MarzoCompras_MD_B2_20190704145959_00856.jpg and waiting for 1
Worker 6: Status:{'status': 'Succeeded', '
Worker 6: Finished 496.MarzoCompras_MD_B2_20190704143331_00153.jpg and waiting for 1
Worker 8: Reading 496.MarzoCompras_MD_B2_20190704172514_00558.jpg
Worker 0: Reading 495.MarzoCompras_LV_B4_20190708124104_00754.jpg
Worker 9: Status:{'status': 'Succeeded', '
Worker 9: Finished 492.MarzoCompras_MD_B2_20190705105251_00226.jpg and waiting for 1
Worker 3: Reading 497.MarzoCompras_MD_B2_20190704102515_00876.jpg
Worker 7: Reading 494.MarzoCompras_MD_B2_20190704145959_00868.jpg
Worker 4: Status:{'status': 'Succeeded', '
Worker 4: Finished 493.MarzoCompras_MD_B2_2019070

Worker 7: Reading 498.MarzoCompras_MD_B2_20190704150000_00877.jpg
Worker 9: Status:{'status': 'Succeeded', '
Worker 9: Finished 496.MarzoCompras_MD_B2_20190705105252_00250.jpg and waiting for 1
Worker 4: Status:{'status': 'Succeeded', '
Worker 4: Finished 497.MarzoCompras_MD_B2_20190704112224_00653.jpg and waiting for 1
Worker 9: Reading 497.MarzoCompras_MD_B2_20190705105252_00260.jpg
Worker 4: Reading 498.MarzoCompras_MD_B2_20190704112224_00656.jpg
Worker 5: Status:{'status': 'Succeeded', '
Worker 5: Finished 498.MarzoCompras_MD_B2_20190704123653_00370.jpg and waiting for 1
Worker 1: Status:{'status': 'Succeeded', '
Worker 1: Finished 495.MarzoCompras_LV_B4_20190708164700_00441.jpg and waiting for 1
Worker 2: Status:{'status': 'Succeeded', '
Worker 2: Finished 489.MarzoCompras_MD_B2_20190704095257_00036.jpg and waiting for 1
Worker 5: Reading 499.MarzoCompras_MD_B2_20190704123653_00373.jpg
Worker 0: Status:{'status': 'Succeeded', '
Worker 0: Finished 499.MarzoCompras_LV_B4_20190708124

In [102]:
# MARZO COMPRAS
traslation_dict = {
    'dni':{'tras_x':27,'tras_y':-44,'width':340,'height':75},
    'tel':{'tras_x':27+455,'tras_y':-44,'width':290,'height':75},
    'mail':{'tras_x':60,'tras_y':185,'width':800,'height':65},
    'nomb_ape1':{'tras_x':240,'tras_y':-180,'width':540,'height':80},
    'nomb_ape2':{'tras_x':-40,'tras_y':-105,'width':820,'height':65},
    'dir1':{'tras_x':100,'tras_y':20,'width':670,'height':65},
    'dir2':{'tras_x':-40,'tras_y':72,'width':820,'height':65},
    'distrito':{'tras_x':70,'tras_y':127,'width':800,'height':65},
    'cc':{'tras_x':200,'tras_y':240,'width':800,'height':65}
}

lines_dict = {
    'dni_tel':{'tras_x':-40,'tras_y':-45,'width':820,'height':80}
}
eqs = {
    'Normal':90.00,
    'Low':70.00
}

banned_field_dict = ['DNI:','DNI','DN','DN1']+['Telefono:','Telefono']+['E-mail:']+['Nombres y Apellidos:']+['Direccion:']+['Distrito:']+['Centro Comercial:']

n_eqs = {'%':'90',')':'',',':'','$':'5','f':'7','&':'6','+':'7','g':'9','l':'1','o':'0','y':'4','/':'1','.':'','-':''}


def center_of(bounding_box):
    xs = np.array([bounding_box[0],bounding_box[2],bounding_box[4],bounding_box[6]])
    ys = np.array([bounding_box[1],bounding_box[3],bounding_box[5],bounding_box[7]])
    return int(xs.sum() / xs.size) , int(ys.sum() / ys.size)

def dni_rect_from_ce(x, y, cv2_v, img, areas_dict, lines_dict):
    w = img.shape[1]
    h = img.shape[0]

    color = (0, 0, 255)
    thicknes = 2

    fields = []
    res_dict = {}

    for key in areas_dict.keys():
        tras_vector = np.array([areas_dict[key]['tras_x'], areas_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([areas_dict[key]['width'], areas_dict[key]['height']])

        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]

        cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)
        # print('{} {}'.format(point,point_2))

    for key in lines_dict.keys():
        tras_vector = np.array([lines_dict[key]['tras_x'], lines_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([lines_dict[key]['width'], lines_dict[key]['height']])

        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]

        # cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)

    # return res_dict['dni'][0], res_dict['dni'][1], res_dict['tel'][0], res_dict['tel'][1]
    return res_dict

def is_in_box(center, top_left_point, bottom_right_point):
    x = center[0] > top_left_point[0] and center[0] < bottom_right_point[0]
    y = center[1] > top_left_point[1] and center[1] < bottom_right_point[1]
    return (x and y)

def get_fields_from_json(azure_json, areas_dict):
    fields = {}
    scores = {}
    j=0

    # Setting empty arrays in result
    for key in areas_dict.keys():
        fields[key] = []
        scores[key] = []

    for line in azure_json['recognitionResult']['lines']:
        j+=1
        # print('Line {}: {},{}'.format(j,line['text'],line['boundingBox']))

        # Drawing areas
        cv2.rectangle(image,(line['boundingBox'][0],line['boundingBox'][1]),(line['boundingBox'][2],line['boundingBox'][5]),(0,0,0),1)
        cv2.circle(image,center_of(line['boundingBox']), 4, (0,255,255), -1)

        # Searching in areas_dict
        c = center_of(line['boundingBox'])
        t = line['text']
        ws = []
        scos = []
        for w in line['words']:
            ws.append(w['text'])
            try:
                scos.append(w['confidence'])
            except:
                scos.append('Normal')

        for key in areas_dict.keys():
            # print('C {} está en {}?'.format(c, areas_dict[key]))
            if is_in_box(c, areas_dict[key][0], areas_dict[key][1]):
                # print('String: {} --> {}'.format(t,key))
                fields[key].append(ws)
                scores[key].append(scos)

    return fields, scores


def basic_digit_clean(string, num_eqs):
    for k in num_eqs.keys():
        string = string.replace(k, num_eqs[k])
    return string


def basic_field_clean(string):
    a = string.find(':')
    if a == len(string)-1:
        return string[:-1]
    else:
        if a >= 0:
            return string[a + 1:]
        return string

def post_num_field(l_field, l_score, banned, num_eqs={}, separator=""):
    value = ""
    score = 0.0
    ct = 0

    for i in range(len(l_field)):
        for j in range(len(l_field[i])):
            # print(j)
            if l_field[i][j] not in banned:
                ct += 1
                value += separator + l_field[i][j]
                # print('{}'.format(l_score[i][j]))
                score += eqs[l_score[i][j]]
    value = basic_digit_clean(value, num_eqs)
    return (value.strip(), score/ct) if ct > 0 else (value, 0.0)


def get_dni_from_line(string, num_eqs):
    a = string.find('DNI:')
    b = string.find('Telefono:')
    if a>=0 and b>=0:
        return basic_digit_clean(string[a+len('DNI:'):b],num_eqs)
    a = string.find('DNI')
    b = string.find('Telefono')
    if a>=0 and b>=0:
        return basic_digit_clean(string[a+len('DNI'):b],num_eqs)
    else:
        return ""

def get_tel_from_line(string, num_eqs):
    b = string.find('Telefono:')
    if b>=0:
        return basic_digit_clean(string[b+len('Telefono:'):],num_eqs)
    b = string.find('Telefono')
    if b>=0:
        return basic_digit_clean(string[b+len('Telefono'):],num_eqs)
    else:
        return ""

In [89]:
'3453453540'.find('0') ==  len('3453453540') - 1

True

In [ ]:
basic_field_clean('')

In [96]:
# MarzoCompras_XM_B5_20190709180636_00948, MarzoCompras_XM_B5_20190709154959_00855
filenames = []
for i in range(azure_files.size):
    n = azure_files[i].split('\\')[-1]
    filenames.append(n)
filenames = np.array(filenames)

In [97]:
np.argwhere(filenames=='MarzoCompras_XM_B5_20190709180636_00948.jpg')[0][0]

3001

In [101]:
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\'
idx = np.argwhere(filenames=='MarzoCompras_XM_B5_20190710103950_00558.jpg')[0][0]

# for i in range(azure_jsons.size):
for i in range(idx,idx+1):
    # Getting json info
    j=0
    # Reading the image
    name = azure_files[i].split('\\')[-1]
    print('Para: {}'.format(azure_files[i]))
    image = cv2.imread(ruta + name)
    
    for line in azure_jsons[i]['recognitionResult']['lines']:
        # final_str += line['text']
        j+=1
        # print('Line {}: {},{}'.format(j,line['text'],line['boundingBox']))
        print('Line {}:'.format(line['text']))
        for word in line['words']:
            print('{} {}'.format(word['text'], word['boundingBox']))
            cv2.rectangle(image,(word['boundingBox'][0],word['boundingBox'][1]),(word['boundingBox'][2],word['boundingBox'][5]),(0,0,255),1)
            cv2.circle(image,center_of(word['boundingBox']), 4, (255,0,255), -1)
            cv2.putText(image,'{}'.format(word['text']), (word['boundingBox'][0],word['boundingBox'][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)
        # print('{}'.format(line))
        cv2.rectangle(image,(line['boundingBox'][0],line['boundingBox'][1]),(line['boundingBox'][2],line['boundingBox'][5]),(0,0,0),1)
        cv2.circle(image,center_of(line['boundingBox']), 4, (0,255,255), -1)
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Para: E:\git\cupones_wong\notebooks_flow\input\MarzoCompras_XM_B5_20190710103950_00558.jpg
Line Nombres y Apellidos::
Nombres [39, 44, 154, 43, 154, 78, 39, 74]
y [160, 43, 176, 43, 176, 78, 160, 78]
Apellidos: [182, 43, 309, 42, 310, 79, 182, 78]
Line CCLECO:
CCLECO [350, 18, 553, 29, 553, 75, 345, 68]
Line Bringas:
Bringas [142, 86, 385, 91, 385, 131, 141, 129]
Line Burgos:
Burgos [462, 88, 670, 91, 669, 134, 462, 129]
Line DNI::
DNI: [40, 159, 87, 159, 87, 188, 40, 188]
Line Telefono::
Telefono: [433, 159, 547, 163, 546, 191, 432, 189]
Line 946192065:
946192065 [553, 133, 828, 144, 827, 190, 551, 181]
Line Direccion: Av palmeran:
Direccion: [38, 219, 174, 210, 175, 250, 38, 247]
Av [203, 209, 267, 207, 267, 250, 203, 250]
palmeran [317, 206, 576, 211, 575, 247, 317, 251]
Line 5 64 3:
5 [551, 264, 576, 263, 577, 304, 552, 304]
64 [605, 262, 661, 259, 662, 302, 606, 303]
3 [679, 259, 705, 258, 706, 302, 680, 302]
Line Distrito::
Distrito: [40, 333, 138, 335, 137, 363, 39, 362]
Line Lo

In [76]:
# SHOWING IMAGES
y_min = 120
y_max = 213
x_min = 0
x_max = 833

for i in range(idx,idx+1):
    # Reading the image
    name = azure_files[i].split('\\')[-1]
    print('Para: {}'.format(azure_files[i]))
    image = cv2.imread(ruta + name)
    
    # Grabbing the file
    filename = ruta + name
    name = filename.split('\\')
    name = name[len(name)-1]
    name = name.split('.')[0]
    
    c_x = int(m_cxs[i])
    c_y = int(m_cys[i])
    
    wi = m_widths[i]
    hi = m_heights[i]
    
    x_minimo = int(c_x-(wi/2))
    x_maximo = int(c_x+(wi/2))
    y_minimo = int(c_y-(hi/2))
    y_maximo = int(c_y+(hi/2))
    
    image = cv2.imread(filename)
    found = False
    
    if (c_x==9999 or c_y==9999):
        cv2.putText(image,'{}'.format('Area no detectada'), (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        found = False
    else:
        found = True
        print('{} . {}'.format(i, name))
        
        cupon_areas = dni_rect_from_ce(c_x, c_y+y_min, cv2, image, traslation_dict, lines_dict)
        # print('{}'.format(cupon_areas))
        fields, scores = get_fields_from_json(azure_jsons[i], cupon_areas)
        
        # Parsing numeric fields
        dni, sdni = post_num_field(fields['dni'],scores['dni'],banned_field_dict,n_eqs)
        tel, stel = post_num_field(fields['tel'],scores['tel'],banned_field_dict,n_eqs)
        # Parsing mail
        mail, smail = post_num_field(fields['mail'],scores['mail'],banned_field_dict)
        # Parsing other fields
        nomb1, snomb1 = post_num_field(fields['nomb_ape1'],scores['nomb_ape1'],banned_field_dict,separator=" ")
        nomb2, snomb2 = post_num_field(fields['nomb_ape2'],scores['nomb_ape2'],banned_field_dict,separator=" ")
        dire1, sdire1 = post_num_field(fields['dir1'],scores['dir1'],banned_field_dict,separator=" ")
        dire2, sdire2 = post_num_field(fields['dir2'],scores['dir2'],banned_field_dict,separator=" ")
        dist, sdist = post_num_field(fields['distrito'],scores['distrito'],banned_field_dict,separator=" ")
        cc, scc = post_num_field(fields['cc'],scores['cc'],banned_field_dict,separator=" ")
        # Concatenating two lines fields
        nomb_ape, snomb_ape = nomb1+" "+nomb2, (snomb1+snomb2)/2
        direccion, sdireccion = dire1+" "+dire2, (sdire1+sdire2)/2
        
        # Last trying to get DNI
        if sdni==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            dni = get_dni_from_line(st, n_eqs)
            sdni = 0.0 if dni=="" else sc
        # Last trying to get Tel
        if stel==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            tel = get_tel_from_line(st, n_eqs)
            stel = 0.0 if tel=="" else sc
        
        # Showing fields
        print('DNI {} {}'.format(dni, sdni))
        print('TEL {} {}'.format(tel, stel))
        print('MAIL {} {}'.format(mail, smail))
        print('NOMB {} {}'.format(nomb_ape, snomb_ape))
        # print('NOMB2 {} {}'.format(nomb2, snomb2))
        print('DIR {} {}'.format(direccion, sdireccion))
        # print('DIR2 {} {}'.format(dire2, sdire2))
        print('DIST {} {}'.format(dist, sdist))
        print('CC {} {}'.format(cc, scc))

    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Para: E:\git\cupones_wong\notebooks_flow\input\MarzoCompras-0303072019132605_00529.jpg
48 . MarzoCompras-0303072019132605_00529
DNI 1015397: 87.5
TEL 990442682 90.0
MAIL Im-floressuarez(@hotmail 85.0
NOMB Nombres y Apellidos: Luis Manve Flores Suarez 89.0
DIR 1r. Jos Incas 452  43.75
DIST Independenta 85.0
CC plaza norte . 88.33333333333333


In [103]:
# PROCESSING
y_min = 120
y_max = 213
x_min = 0
x_max = 833

filenames = []

dnis = []
tels = []
mails = []
nombres = []
dirs = []
distris = []
ccs = []

sco_dnis = []
sco_tels = []
sco_mails = []
sco_nombres = []
sco_dirs = []
sco_distris = []
sco_ccs = []

azure_str_jsons = []

for i in range(azure_files.size):
# for i in range(idx,idx+1):
    # Reading the image
    n = azure_files[i].split('\\')[-1]
    # print('Para: {}'.format(azure_files[i]))
    # image = cv2.imread(ruta + name)
    
    filename = ruta + n
    name = filename.split('\\')
    name = name[len(name) - 1]
    name = name.split('.')[0]

    c_x = int(m_cxs[i])
    c_y = int(m_cys[i])

    wi = m_widths[i]
    hi = m_heights[i]

    x_minimo = int(c_x - (wi / 2))
    x_maximo = int(c_x + (wi / 2))
    y_minimo = int(c_y - (hi / 2))
    y_maximo = int(c_y + (hi / 2))

    image = cv2.imread(filename)
    filenames.append(name)

    if (c_x == 9999 or c_y == 9999):
        dnis.append("")
        tels.append("")
        mails.append("")
        nombres.append("")
        dirs.append("")
        distris.append("")
        ccs.append("")

        sco_dnis.append(0.0)
        sco_tels.append(0.0)
        sco_mails.append(0.0)
        sco_nombres.append(0.0)
        sco_dirs.append(0.0)
        sco_distris.append(0.0)
        sco_ccs.append(0.0)
    else:
        print('{}.{}'.format(i, name))

        cupon_areas = dni_rect_from_ce(c_x, c_y+y_min, cv2, image, traslation_dict, lines_dict)
        # print('{}'.format(cupon_areas))
        fields, scores = get_fields_from_json(azure_jsons[i], cupon_areas)

        # Parsing numeric fields
        dni, sdni = post_num_field(fields['dni'],scores['dni'],banned_field_dict,n_eqs)
        tel, stel = post_num_field(fields['tel'],scores['tel'],banned_field_dict,n_eqs)
        # print('Before DNI: {}'.format(dni))
        dni, tel = basic_field_clean(dni), basic_field_clean(tel)
        # print('Afetr clean DNI: {}'.format(dni))

        # Parsing mail
        mail, smail = post_num_field(fields['mail'],scores['mail'],banned_field_dict)
        mail = basic_field_clean(mail)

        # Parsing other fields
        nomb1, snomb1 = post_num_field(fields['nomb_ape1'],scores['nomb_ape1'],banned_field_dict,separator=" ")
        nomb2, snomb2 = post_num_field(fields['nomb_ape2'],scores['nomb_ape2'],banned_field_dict,separator=" ")
        dire1, sdire1 = post_num_field(fields['dir1'],scores['dir1'],banned_field_dict,separator=" ")
        dire2, sdire2 = post_num_field(fields['dir2'],scores['dir2'],banned_field_dict,separator=" ")
        dist, sdist = post_num_field(fields['distrito'],scores['distrito'],banned_field_dict,separator=" ")
        cc, scc = post_num_field(fields['cc'],scores['cc'],banned_field_dict,separator=" ")

        nomb1 = basic_field_clean(nomb1)
        nomb2 = basic_field_clean(nomb2)
        dire1 = basic_field_clean(dire1)
        dire2 = basic_field_clean(dire2)
        dist = basic_field_clean(dist)
        cc = basic_field_clean(cc)

        # Concatenating two lines fields
        nomb_ape, snomb_ape = nomb1+" "+nomb2, (snomb1+snomb2)/2
        direccion, sdireccion = dire1+" "+dire2, (sdire1+sdire2)/2

        # Last trying to get DNI
        if sdni==0:
            print('Trying to get DNI')
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            print('{}'.format(fields['dni_tel']))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            dni = get_dni_from_line(st, n_eqs)
            sdni = 0.0 if dni=="" else sc
        # Last trying to get Tel
        if stel==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            tel = get_tel_from_line(st, n_eqs)
            stel = 0.0 if tel=="" else sc

        # Showing fields
        print('DNI {} {}'.format(dni, sdni))
        print('TEL {} {}'.format(tel, stel))
        # print('MAIL {} {}'.format(mail, smail))
        # print('NOMB {} {}'.format(nomb_ape, snomb_ape))
        # print('NOMB2 {} {}'.format(nomb2, snomb2))
        # print('DIR {} {}'.format(direccion, sdireccion))
        # print('DIR2 {} {}'.format(dire2, sdire2))
        # print('DIST {} {}'.format(dist, sdist))
        # print('CC {} {}'.format(cc, scc))

        # Appending into arrays
        dnis.append(dni)
        tels.append(tel)
        mails.append(mail)
        nombres.append(nomb_ape)
        dirs.append(direccion)
        distris.append(dist)
        ccs.append(cc)

        sco_dnis.append(sdni)
        sco_tels.append(stel)
        sco_mails.append(smail)
        sco_nombres.append(snomb_ape)
        sco_dirs.append(sdireccion)
        sco_distris.append(sdist)
        sco_ccs.append(scc)

        azure_str_jsons.append(json.dumps(azure_jsons[i]))

print('Termino :D ')

0.MarzoCompras-0303072019121007_00008
DNI 100672966 80.0
TEL 98233140 70.0
1.MarzoCompras-0303072019121007_00010
DNI 09883016 70.0
TEL 946498591 78.0
2.MarzoCompras-0303072019121007_00025
DNI 04015880 80.0
TEL 92848937 70.0
3.MarzoCompras-0303072019121013_00265
DNI 09310742 76.66666666666667
TEL 985973705 70.0
4.MarzoCompras-0303072019121014_00331
DNI 08281361949352892 76.66666666666667
TEL 949352892 78.0
5.MarzoCompras-0303072019121014_00343
Trying to get DNI
[['DNI:'], ['of', 'yoly', '/', 'Telefono:', '536', '765.']]
('DNI:ofyoly/Telefono:536765.', 78.57142857142857)
DNI 0740141 78.57142857142857
TEL 536765 78.57142857142857
6.MarzoCompras-0303072019121017_00450
DNI DN09900151 70.0
TEL 916881398 70.0
7.MarzoCompras-0303072019121017_00459
DNI 08982279 75.0
TEL 995327405 82.0
8.MarzoCompras-0303072019121018_00497
DNI 45277490 75.0
TEL 75521928 70.0
9.MarzoCompras-0303072019121018_00501
DNI 05333314 70.0
TEL 999171709 70.0
10.MarzoCompras-0303072019121019_00514
DNI 071546720 70.0
TEL 97

DNI 43351631 90.0
TEL 945682565 70.0
88.MarzoCompras-0303072019164223_00407
DNI 10497518 80.0
TEL 954786608 70.0
89.MarzoCompras-0303072019164225_00499
DNI 27988142 83.33333333333333
TEL 982612749 80.0
90.MarzoCompras-0303072019164226_00511
DNI 279881491 70.0
TEL 98261224 74.0
91.MarzoCompras-0303072019164227_00551
DNI 45038476 80.0
TEL 952388518 86.0
92.MarzoCompras-0303072019164227_00562
DNI 48064827 83.33333333333333
TEL 975327693 90.0
93.MarzoCompras-0303072019164229_00674
DNI 45795971 70.0
TEL 93644462 83.33333333333333
94.MarzoCompras-0303072019164229_00675
DNI 44822698 85.0
TEL 970378345 70.0
95.MarzoCompras-0303072019164230_00714
Trying to get DNI
[['DNI:'], ['468', '50', '7', '10', 'Telefono:', '753585415']]
('DNI:46850710Telefono:753585415', 75.71428571428571)
DNI 46850710 75.71428571428571
TEL 753585415 75.71428571428571
96.MarzoCompras-0303072019164230_00719
Trying to get DNI
[['DNI:', '466', '82', '940', 'Telefono:', '94', '35', '24335']]
('DNI:46682940Telefono:943524335',

180.MarzoCompras_0203072019130747_00526
Trying to get DNI
[['DNI:', '3', '2', '8', '1', '/', '2', '50', 'Telefono:', '043-', '3298', '35']]
('DNI:3281/250Telefono:043-329835', 80.0)
DNI 32811250 80.0
TEL 043329835 80.0
181.MarzoCompras_0203072019130748_00570
DNI 101386CZ5374752 73.33333333333333
TEL 5374752 75.0
182.MarzoCompras_0203072019130749_00573
Trying to get DNI
[['DNI:'], ['Telefono:'], ['89752', '17JJ']]
('DNI:Telefono:8975217JJ', 80.0)
DNI  0.0
TEL 8975217JJ 70.0
183.MarzoCompras_0203072019130749_00598
DNI 46611060 83.33333333333333
TEL 971329518 70.0
184.MarzoCompras_0203072019130749_00605
DNI 10629548 90.0
TEL 977445290 80.0
185.MarzoCompras_0203072019130751_00648
DNI 06296898983982767 75.0
TEL 983982767 73.33333333333333
186.MarzoCompras_0203072019130751_00651
DNI 10431321 83.33333333333333
TEL 945321160 83.33333333333333
187.MarzoCompras_0203072019130752_00699
DNI 43236040 80.0
TEL 964324692 70.0
188.MarzoCompras_AV_B3_20190705171100_00011
Trying to get DNI
[['DNI:'], ['1

276.MarzoCompras_AV_B3_20190708103529_00029
DNI 70624447 90.0
TEL 931279508 90.0
277.MarzoCompras_AV_B3_20190708103529_00036
DNI 41678064 70.0
TEL 995487187 80.0
278.MarzoCompras_AV_B3_20190708103530_00076
DNI 10153975 70.0
TEL 90442632 70.0
279.MarzoCompras_AV_B3_20190708103530_00093
DNI 47283769 80.0
TEL 992252341 83.33333333333333
280.MarzoCompras_AV_B3_20190708103531_00107
Trying to get DNI
[['DNI:'], ['0909483', '/', 'Telefono:', '279769419']]
('DNI:0909483/Telefono:279769419', 78.0)
DNI 09094831 78.0
TEL 279769419 78.0
281.MarzoCompras_AV_B3_20190708103532_00147
DNI 47927744 70.0
TEL 39640z4 70.0
282.MarzoCompras_AV_B3_20190708103532_00155
Trying to get DNI
[['DNI:', '000', '50', '937', 'Telefono:', '93', '7', '563', '15']]
('DNI:00050937Telefono:93756315', 83.33333333333333)
DNI 00050937 83.33333333333333
TEL 93756315 83.33333333333333
283.MarzoCompras_AV_B3_20190708103535_00284
Trying to get DNI
[['DNI:'], ['Telefono:', '906207081']]
('DNI:Telefono:906207081', 83.33333333333333

DNI 10025290 90.0
TEL 95994809 90.0
372.MarzoCompras_AV_B4_20190708180945_00799
DNI 75228737 90.0
TEL 955317109 85.0
373.MarzoCompras_AV_B4_20190708180946_00843
DNI 76848149 70.0
TEL 950273308 70.0
374.MarzoCompras_AV_B4_20190708180947_00859
Trying to get DNI
[['Telefono:'], ['98', '73', '1', '1423']]
('Telefono:987311423', 82.0)
DNI  0.0
TEL 987311423 80.0
375.MarzoCompras_AV_B4_20190708180948_00923
DNI (3579002 80.0
TEL 9901132603 80.0
376.MarzoCompras_AV_B4_20190709094718_00006
DNI 44768976 70.0
TEL 950969134 90.0
377.MarzoCompras_AV_B4_20190709094719_00030
Trying to get DNI
[['DNI:', '092', '9', '3', '9', '94', 'Telefono:', '9', '96', '69', '24-13']]
('DNI:09293994Telefono:9966924-13', 79.0909090909091)
DNI 09293994 79.0909090909091
TEL 996692413 79.0909090909091
378.MarzoCompras_AV_B4_20190709094719_00050
DNI 40471113 70.0
TEL 953299308 80.0
379.MarzoCompras_AV_B4_20190709094721_00089
DNI 43125948 90.0
TEL 941713491 70.0
380.MarzoCompras_AV_B4_20190709094721_00103
DNI 43125948 70.

...

2780.MarzoCompras_XM_B3_20190705180532_01011
DNI H731332 70.0
TEL 942686498 83.33333333333333
2781.MarzoCompras_XM_B3_20190708094653_00042
DNI 80312958 82.0
TEL 90888176 86.0
2782.MarzoCompras_XM_B3_20190708094654_00044
Trying to get DNI
[['DNI:'], ['80', '312', '95', 'X', 'Telefono', ',', '79058x/76']]
('DNI:8031295XTelefono,79058x/76', 75.0)
DNI :8031295X 75.0
TEL 79058x176 75.0
2783.MarzoCompras_XM_B3_20190708094659_00062
DNI 09049919 80.0
TEL 59952260 70.0
2784.MarzoCompras_XM_B3_20190708094716_00126
DNI 75816510 80.0
TEL 997065433 90.0
2785.MarzoCompras_XM_B3_20190708094717_00130
DNI 72446130 70.0
TEL 987824120 70.0
2786.MarzoCompras_XM_B3_20190708094729_00176
DNI 25573949 82.0
TEL 999912267 86.66666666666667
2787.MarzoCompras_XM_B3_20190708094742_00229
DNI 44741096 80.0
TEL 160877909 80.0
2788.MarzoCompras_XM_B3_20190708094818_00325
DNI 074077896026220 75.0
TEL 96026220 76.0
2789.MarzoCompras_XM_B3_20190708094820_00333
Trying to get DNI
[['DNI:'], ['1074', '8050', 'Telefono:', '7

DNI 08781214Te1e70n0 80.0
TEL 996984304 83.33333333333333
2868.MarzoCompras_XM_B4_20190708153352_00862
Trying to get DNI
[['DNI'], ['3292', '5737', 'Telefono:/', '33450811']]
('DNI32925737Telefono:/33450811', 86.0)
DNI 32925737 86.0
TEL 133450811 86.0
2869.MarzoCompras_XM_B4_20190708153355_00934
DNI 0603153Te1e70n0 70.0
TEL 990011816 83.33333333333333
2870.MarzoCompras_XM_B4_20190708153357_00986
DNI 09698644 70.0
TEL 537286077 76.66666666666667
2871.MarzoCompras_XM_B4_20190708153357_01011
DNI 48312943 80.0
TEL 933064165 83.33333333333333
2872.MarzoCompras_XM_B4_20190708153358_01020
DNI 73033783 70.0
TEL 2814012 80.0
2873.MarzoCompras_XM_B4_20190708153358_01039
DNI 323576Te1e70n0: 70.0
TEL  0.0
2874.MarzoCompras_XM_B4_20190708153359_01046
Trying to get DNI
[['DNI:', '0', '85', '900', '0', '2', 'Telefono:', '98', '95', '0', '1/23']]
('DNI:08590002Telefono:989501/23', 79.0909090909091)
DNI 08590002 79.0909090909091
TEL 989501123 79.0909090909091
2875.MarzoCompras_XM_B4_20190708165815_0010

DNI 096136Octe1e70n0 70.0
TEL 998770354 70.0
2945.MarzoCompras_XM_B5_20190709154954_00771
DNI 06281061 70.0
TEL 949078849 70.0
2946.MarzoCompras_XM_B5_20190709154957_00824
DNI 47959564 80.0
TEL 971922730 82.0
2947.MarzoCompras_XM_B5_20190709154958_00834
DNI 09281361 70.0
TEL 949357892 70.0
2948.MarzoCompras_XM_B5_20190709154959_00855
DNI 0909483122769418 75.0
TEL 22769418 73.33333333333333
2949.MarzoCompras_XM_B5_20190709155000_00876
DNI 08623952 85.0
TEL 941714113 70.0
2950.MarzoCompras_XM_B5_20190709155000_00883
DNI 46371401 83.33333333333333
TEL 593410766 80.0
2951.MarzoCompras_XM_B5_20190709155001_00901
DNI 41743307 70.0
TEL 714148203 70.0
2952.MarzoCompras_XM_B5_20190709155001_00913
DNI 40706730 90.0
TEL 960420047 90.0
2953.MarzoCompras_XM_B5_20190709155006_00975
DNI 52384953 80.0
TEL 984266458 85.0
2954.MarzoCompras_XM_B5_20190709155007_00993
DNI 70506491 70.0
TEL 949567042 70.0
2955.MarzoCompras_XM_B5_20190709163304_00093
DNI 40361153 70.0
TEL 946890694 70.0
2956.MarzoCompras_XM

3040.MarzoCompras_XM_B5_20190710100552_00614
DNI 435586204737367 86.66666666666667
TEL 4737367 85.71428571428571
3041.MarzoCompras_XM_B5_20190710100554_00663
DNI 40503163 90.0
TEL 99752988 70.0
3042.MarzoCompras_XM_B5_20190710100555_00686
DNI 41513921 80.0
TEL 5424070 70.0
3043.MarzoCompras_XM_B5_20190710100555_00687
DNI 08433413 70.0
TEL Te1e70n0989072718 75.0
3044.MarzoCompras_XM_B5_20190710100559_00742
DNI 07930475 70.0
TEL 937615016 70.0
3045.MarzoCompras_XM_B5_20190710100559_00761
DNI 07633047989859488 85.0
TEL 989859488 80.0
3046.MarzoCompras_XM_B5_20190710100602_00830
DNI 07886807 70.0
TEL 940610450 76.66666666666667
3047.MarzoCompras_XM_B5_20190710100606_00930
DNI 09138120 90.0
TEL 2780841 70.0
3048.MarzoCompras_XM_B5_20190710100606_00935
DNI 09138120 85.0
TEL 278081 83.33333333333333
3049.MarzoCompras_XM_B5_20190710103935_00108
DNI 73211277 76.66666666666667
TEL 933360591 80.0
3050.MarzoCompras_XM_B5_20190710103946_00439
DNI 10378298 90.0
TEL 943421764 70.0
3051.MarzoCompras_X

DNI 42751535 70.0
TEL 978004066 70.0
3136.MarzoCompras_XM_B5_20190710134536_00590
DNI 01137544 70.0
TEL 982053288 80.0
3137.MarzoCompras_XM_B5_20190710134537_00597
DNI 1669655 74.0
TEL 54367406 70.0
3138.MarzoCompras_XM_B5_20190710134537_00607
Trying to get DNI
[['DNI:'], ['(6', '69', ')', '65S', 'Telefono:', '943', '674106']]
('DNI:(669)65STelefono:943674106', 75.0)
DNI (66965S 75.0
TEL 943674106 75.0
3139.MarzoCompras_XM_B5_20190710134538_00628
Trying to get DNI
[['DNI:'], ['10206157', 'Telefono:', '9191', '25', '1047']]
('DNI:10206157Telefono:9191251047', 76.66666666666667)
DNI 10206157 76.66666666666667
TEL 9191251047 76.66666666666667
3140.MarzoCompras_XM_B5_20190710134539_00654
DNI 41942646 80.0
TEL 972784712 70.0
3141.MarzoCompras_XM_B5_20190710134539_00656
DNI 41042646 70.0
TEL 97784172 70.0
3142.MarzoCompras_XM_B5_20190710134548_00802
DNI 70333304 80.0
TEL 987782790 70.0
3143.MarzoCompras_XM_B5_20190710134548_00811
DNI 080216390 76.66666666666667
TEL 962912444 70.0
3144.MarzoC

In [104]:
bd_azure = pd.DataFrame({'NombreArchivo': np.array(filenames)})
# bd_azure['idCupon'] = np.arange(1000, 1000+np.array(dnis).size,1)
bd_azure['DNI'] = np.array(dnis)
bd_azure['AcertividadDNI'] = np.array(sco_dnis)

bd_azure['Telefono'] = np.array(tels)
bd_azure['AcertividadTelefono'] = np.array(sco_tels)

bd_azure['NombreCompleto'] = np.array(nombres)
bd_azure['AcertividadNombreCompleto'] = np.array(sco_nombres)

bd_azure['Direccion'] = np.array(dirs)
bd_azure['AcertividadDireccion'] = np.array(sco_dirs)

bd_azure['Distrito'] = np.array(distris)
bd_azure['AcertividadDistrito'] = np.array(sco_distris)

bd_azure['Correo'] = np.array(mails)
bd_azure['AcertividadCorreo'] = np.array(sco_mails)

# bd_azure['AzureJsonOCR'] = np.array(azure_str_jsons)
# bd_azure['idBatch'] = int(dt_string)

bd_azure.to_csv('azure_result/azure_result_metrics_afterclean.csv', encoding='utf-8', index=False)
print('Termino escritura')

Termino escritura
